In [ ]:
%pylab inline
style.use('http://johannesfeist.eu/misc/jf.mplstyle')
np.set_printoptions(linewidth=200)

In [ ]:
from qutip import *

In [ ]:
from jftools.short_iterative_lanczos import lanczos_timeprop

In [ ]:
from scipy.sparse import lil_matrix
N = 20000
H = lil_matrix((N,N))
H[range(N),range(N)] = -2.
H[range(N-1),range(1,N)] = 1
H[range(1,N),range(N-1)] = 1
H = H.tocsr()

In [ ]:
phi0 = exp(-(arange(N)-15000)**2/(2*300**2)-1.5j*arange(N)) + exp(-(arange(N)-5000)**2/(2*50**2)+1j*arange(N))
phi0 /= norm(phi0)
phir = randn(N).astype(complex)
phir /= norm(phir)

In [ ]:
Hq = Qobj(H)
phi0q = Qobj(phi0)
phirq = Qobj(phir)

In [ ]:
prop = lanczos_timeprop(H,12,1e-10)
prop.propagate(phi0,[0,.5])
prop.propagate(phir,[0,.5])

In [ ]:
from scipy.sparse.linalg import expm_multiply
dt = 0.5
prop = lanczos_timeprop(H,12,1e-12)
phi1 = prop.propagate(phi0,[0,dt])[-1]
phi2 = expm_multiply(-1j*dt*H,phi0)
np.allclose(phi1,phi2)

In [ ]:
%timeit lanczos_timeprop(H,12,1e-12).propagate(phi0,[0.,dt])
prop = lanczos_timeprop(H,12,1e-12)
%timeit prop.propagate(phi0,[0.,dt])
prop.phia[0][:] = phi0
%timeit prop._step(0,dt)
%timeit phi2 = expm_multiply(-1j*dt*H,phi0)
%timeit sesolve(Hq,phi0q,[0.,dt],[])

In [ ]:
ts = linspace(0,100,11)
prop = lanczos_timeprop(H,12,1e-12)
for p0 in phi0,phir:
    p0q = Qobj(p0)
    %time phiLs = prop.propagate(p0,ts)
    %time resQ = sesolve(Hq,p0q,ts,[],options=Options(atol=1e-10,rtol=1e-10))
    phiLs = array(phiLs)
    phiQs = array([s.full() for s in resQ.states]).squeeze()
    print(norm(phiLs), norm(phiQs-phiLs), norm(phiQs[-1]-phiLs[-1]))
    f,axs = subplots(1,2,figsize=(8,4),sharex=True,sharey=True)
    axs[0].pcolormesh(ts,arange(N),abs(phiQs.T)**2)
    axs[1].pcolormesh(ts,arange(N),abs(phiLs.T)**2)
    axs[0].autoscale(tight=True)
    f.tight_layout(pad=0.5)

In [ ]:
convgs = [1e-6,1e-8,1e-10,1e-12,1e-14,1e-16]
refphi = lanczos_timeprop(H,12,1e-14).propagate(phir,ts)[-1]

qsolve = lambda tol: array([s.full() for s in sesolve(Hq,phirq,ts,[],options=Options(atol=tol,rtol=tol)).states]).squeeze()
phiLs_convg, phiQs_convg = {}, {}
for c in convgs:
    if c>=1e-14:
        print('SIL',c,end=' ')
        tt = %timeit -qo -n1 -r1 phiLs_convg[c] = lanczos_timeprop(H,12,c).propagate(phir,ts)
        phif = phiLs_convg[c][-1]
        print("%.4f %.4e"%(tt.best,mean(abs(refphi-phif)/abs(refphi))))
    print('RK ',c,end=' ')
    tt = %timeit -qo -n1 -r1 phiQs_convg[c] = qsolve(c)
    phif = phiQs_convg[c][-1]
    print("%.4f %.4e"%(tt.best,mean(abs(refphi-phif)/abs(refphi))))

In [ ]:
def err(a,b,ea,eb):
    dd = dict(Q=phiQs_convg,L=phiLs_convg)
    phia = dd[a][ea][-1]
    phib = dd[b][eb][-1]
    return 2*abs(phia-phib)/(abs(phia)+abs(phib))
def ploterr(a,b,ea,eb):
    relerr = err(a,b,ea,eb)
    names = dict(Q='RK',L='SIL')
    plot(relerr,label="%s %g vs %s %g"%(names[a],ea,names[b],eb))
ploterr('Q','Q',1e-16,1e-06)
ploterr('L','L',1e-14,1e-06)
ploterr('L','L',1e-14,1e-12)
ploterr('Q','L',1e-16,1e-14)
ploterr('L','L',1e-14,1e-10)
ploterr('L','L',1e-14,1e-8)
yscale('log')
ylabel('rel. error')
legend(frameon=True,fontsize=14);

In [ ]:
meanerrsLL = array([(e,mean(err('L','L',1e-14,e))) for e in [1e-6,1e-8,1e-10,1e-12]]).T
meanerrsQL = array([(e,mean(err('Q','L',1e-16,e))) for e in [1e-6,1e-8,1e-10,1e-12,1e-14]]).T
meanerrsLQ = array([(e,mean(err('L','Q',1e-14,e))) for e in [1e-6,1e-8,1e-10,1e-12,1e-14,1e-16]]).T
meanerrsQQ = array([(e,mean(err('Q','Q',1e-16,e))) for e in [1e-6,1e-8,1e-10,1e-12,1e-14]]).T
plot(meanerrsLL[0],meanerrsLL[1],'o-',label='LL')
plot(meanerrsQQ[0],meanerrsQQ[1],'o-',label='QQ')
plot(meanerrsQL[0],meanerrsQL[1],'o--',label='QL')
plot(meanerrsLQ[0],meanerrsLQ[1],'o--',label='LQ')
legend()
yscale('log')
xscale('log')